# Project - Airline AI Assistant

We'll now bring together what we've learned to make an AI Customer Support assistant for an Airline

In [1]:
# imports

import os
import json
from dotenv import load_dotenv
from openai import OpenAI
import gradio as gr

In [2]:
# Initialization

load_dotenv(override=True)

openai_api_key = os.getenv('OPENAI_API_KEY')
if openai_api_key:
    print(f"OpenAI API Key exists and begins {openai_api_key[:8]}")
else:
    print("OpenAI API Key not set")
    
MODEL = "gpt-4o-mini"
openai = OpenAI()

# As an alternative, if you'd like to use Ollama instead of OpenAI
# Check that Ollama is running for you locally (see week1/day2 exercise) then uncomment these next 2 lines
# MODEL = "llama3.2"
# openai = OpenAI(base_url='http://localhost:11434/v1', api_key='ollama')


OpenAI API Key exists and begins sk-proj-


In [93]:
system_message = "You are a helpful assistant for an Airline called FlightAI. "
system_message += "Give short, courteous answers, no more than 1 sentence. "
# system_message += "Just return an existing city then get its' ticket price. "
system_message += "Always be accurate. If you don't know the answer, say so."

In [94]:
# This function looks rather simpler than the one from my video, because we're taking advantage of the latest Gradio updates

def chat(message, history):
    messages = [{"role": "system", "content": system_message}] + history + [{"role": "user", "content": message}]
    response = openai.chat.completions.create(model=MODEL, messages=messages)
    return response.choices[0].message.content

gr.ChatInterface(fn=chat, type="messages").launch()

* Running on local URL:  http://127.0.0.1:7880

To create a public link, set `share=True` in `launch()`.


## Tools

Tools are an incredibly powerful feature provided by the frontier LLMs.

With tools, you can write a function, and have the LLM call that function as part of its response.

Sounds almost spooky.. we're giving it the power to run code on our machine?

Well, kinda.

In [95]:
# Let's start by making a useful function

ticket_prices = {"london": "$799", "paris": "$899", "tokyo": "$1400", "berlin": "$499", "new york": "$1200"}

def get_ticket_price(destination_city):
    print(f"Tool get_ticket_price called for {destination_city}")
    city = destination_city.lower()
    return ticket_prices.get(city, "Unknown")

In [96]:
get_ticket_price("New York")

Tool get_ticket_price called for New York


'$1200'

In [97]:
get_ticket_price("Berlin")

Tool get_ticket_price called for Berlin


'$499'

## LLM專用Dictionary格式，給 **Tool Function Calling（函式調用）**用

In [98]:
# There's a particular dictionary structure that's required to describe our function:

price_function = {
    "name": "get_ticket_price",
    "description": "Get the price of a return ticket to the destination city. Call this whenever you need to know the ticket price, for example when a customer asks 'How much is a ticket to this city'",
    "parameters": {
        "type": "object",
        "properties": {
            "destination_city": {
                "type": "string",
                "description": "The city that the customer wants to travel to",
            },
        },
        "required": ["destination_city"],
        "additionalProperties": False
    }
}

In [99]:
# And this is included in a list of tools:

tools = [{"type": "function", "function": price_function}]

## Getting OpenAI to use our Tool

There's some fiddly stuff to allow OpenAI "to call our tool"

What we actually do is give the LLM the opportunity to inform us that it wants us to run the tool.

Here's how the new chat function looks:

In [100]:
def chat(message, history):
    messages = [{"role": "system", "content": system_message}] + history + [{"role": "user", "content": message}]
    response = openai.chat.completions.create(model=MODEL, messages=messages, tools=tools)

    if response.choices[0].finish_reason == "tool_calls":
        message = response.choices[0].message
        tool_responses, cities = handle_tool_call(message)
        
        messages.append(message)  # 加入 LLM 發出的 tool_call
        for tool_response in tool_responses:
            print(f"Tool Response: {tool_response}")
            messages.append(tool_response)  # 加入每個工具回應
        
        response = openai.chat.completions.create(model=MODEL, messages=messages)

    return response.choices[0].message.content

In [101]:
# We have to write that function handle_tool_call:

def handle_tool_call(message):
    responses = []
    cities = []
    
    for tool_call in message.tool_calls:
        arguments = json.loads(tool_call.function.arguments)
        city = arguments.get('destination_city')
        price = get_ticket_price(city)
        
        response = {
            "role": "tool",
            "content": json.dumps({"destination_city": city, "price": price}),
            "tool_call_id": tool_call.id
        }
        responses.append(response)
        cities.append(city)

    print(f"\nDestination city is: {cities[-1]}")
    return responses, cities


In [102]:
gr.ChatInterface(fn=chat, type="messages").launch()

* Running on local URL:  http://127.0.0.1:7881

To create a public link, set `share=True` in `launch()`.


Tool get_ticket_price called for Paris

Destination city is: Paris
Tool Response: {'role': 'tool', 'content': '{"destination_city": "Paris", "price": "$899"}', 'tool_call_id': 'call_eq1iD3MBxnA3lIYHeIs5T7n5'}
Tool get_ticket_price called for New York

Destination city is: New York
Tool Response: {'role': 'tool', 'content': '{"destination_city": "New York", "price": "$1200"}', 'tool_call_id': 'call_krCxKMsXwRnR0fVZURXClaWq'}
Tool get_ticket_price called for New York

Destination city is: New York
Tool Response: {'role': 'tool', 'content': '{"destination_city": "New York", "price": "$1200"}', 'tool_call_id': 'call_6DxbBUNsA5tq76OOehgucDAD'}


## AI 智慧版城市票價查詢助理

In [107]:
import gradio as gr
import json
from difflib import get_close_matches

import re

def split_cities(destination_text):
    # 正規化所有分隔符：半形空格、全形空格、逗號、中文逗號、頓號
    text = re.sub(r"[,\u3000，、]", " ", destination_text)
    # 再以空白分開
    cities = [city.strip() for city in text.split() if city.strip()]
    return cities

# 票價表
ticket_prices = {
    "london": "$799",
    "paris": "$899",
    "tokyo": "$1400",
    "berlin": "$499",
    "new york": "$1200",
    "taipei": "$1100",
}

# 工具：模糊比對找最接近的城市
def find_closest_city(input_city):
    cities = list(ticket_prices.keys())
    matches = get_close_matches(input_city.lower(), cities, n=1, cutoff=0.6)
    return matches[0] if matches else None

# 工具：取得票價
def get_ticket_price(destination_city):
    print(f"Tool get_ticket_price called for {destination_city}")
    city = destination_city.lower()
    return ticket_prices.get(city, "Unknown")

# 處理工具呼叫 (升級版)
def handle_tool_call(message):
    tool_responses = []

    for tool_call in message.tool_calls:
        arguments = json.loads(tool_call.function.arguments)
        destination_text = arguments.get('destination_city')

        print(f"\nDestination text is: {destination_text}")

        # ✨ 使用新的切分函式
        cities = split_cities(destination_text)
        
        results = []
        unknowns = []

        for city in cities:
            normalized_city = city.lower()
            if normalized_city in ticket_prices:
                price = get_ticket_price(normalized_city)
                results.append({"city": city, "price": price})
            else:
                # 支援找不到城市、模糊比對最接近城市
                closest = find_closest_city(normalized_city)
                if closest:
                    price = get_ticket_price(closest)
                    results.append({"city": closest.title(), "price": price, "note": f"(猜你想找的是 {closest.title()})"})
                else:
                    unknowns.append(city)

        content = {"cities": results, "unknowns": unknowns}

        tool_response = {
            "role": "tool",
            "content": json.dumps(content, ensure_ascii=False),
            "tool_call_id": tool_call.id
        }

        tool_responses.append(tool_response)

    return tool_responses

# 主聊天函式
def chat(message, history):
    messages = [{"role": "system", "content": system_message}] + history + [{"role": "user", "content": message}]
    response = openai.chat.completions.create(model=MODEL, messages=messages, tools=tools)

    if response.choices[0].finish_reason == "tool_calls":
        message = response.choices[0].message
        tool_responses = handle_tool_call(message)

        messages.append(message)  # 加 Assistant tool_call 請求
        messages.extend(tool_responses)  # 加所有 tool responses

        response = openai.chat.completions.create(model=MODEL, messages=messages)

    return response.choices[0].message.content

# system message
system_message = "你是一個票價查詢助理。將輸入的城市轉成英文 再根據工具查到的資料，用清楚簡單的表格回覆使用者。若找不到城市，請友善提示。"

# 定義 tools
price_function = {
    "name": "get_ticket_price",
    "description": "Get the price of a return ticket to one or more destination cities.",
    "parameters": {
        "type": "object",
        "properties": {
            "destination_city": {
                "type": "string",
                "description": "Cities to search, separated by space or comma.",
            },
        },
        "required": ["destination_city"],
        "additionalProperties": False
    }
}

tools = [{"type": "function", "function": price_function}]

# 啟動聊天介面
gr.ChatInterface(fn=chat, type="messages").launch()


* Running on local URL:  http://127.0.0.1:7886

To create a public link, set `share=True` in `launch()`.



Destination text is: Beijing
Tool get_ticket_price called for berlin

Destination text is: Nanjing

Destination text is: Nanjing

Destination text is: Taipei
Tool get_ticket_price called for taipei

Destination text is: Tainan

Destination text is: Taichung

Destination text is: New Taipei City
Tool get_ticket_price called for taipei

Destination text is: New Taipei City
Tool get_ticket_price called for taipei

Destination text is: Taipei
Tool get_ticket_price called for taipei

Destination text is: Beijing
Tool get_ticket_price called for berlin

Destination text is: Tianjin

Destination text is: Shanghai

Destination text is: Taichung
